In [10]:
import pandas as pd
df1 = pd.read_csv('final_data.csv')
df1

,95,48,83,178,72,10,162,42,20,159,176,379,184,70,6,16,187,197,van
0,91,41,84,141,57,9,149,45,19,143,170,330,158,72,9,14,189,199,van
1,104,50,106,209,66,10,207,32,23,158,223,635,220,73,14,9,188,196,saab
2,93,41,82,159,63,9,144,46,19,143,160,309,127,63,6,10,199,207,van
3,85,44,70,205,103,52,149,45,19,144,241,325,188,127,9,11,180,183,bus
4,107,57,106,172,50,6,255,26,28,169,280,957,264,85,5,9,181,183,bus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,93,39,87,183,64,8,169,40,20,134,200,422,149,72,7,25,188,195,saab
841,89,46,84,163,66,11,159,43,20,159,173,368,176,72,1,20,186,197,van
842,106,54,101,222,67,12,222,30,25,173,228,721,200,70,3,4,187,201,saab
843,86,36,78,146,58,7,135,50,18,124,155,270,148,66,0,25,190,195,saab


In [9]:
x = df1.drop('van', axis=1)
x.shape

(845, 18)

In [11]:
y = df1['van'].copy()
y.shape

(845,)

In [12]:
from sklearn.feature_selection import VarianceThreshold

selection = VarianceThreshold(threshold=(0.1))
x = selection.fit_transform(x)
x.shape

(845, 18)

In [14]:
# spliting the training set and testing set
from sklearn.model_selection import train_test_split

xs_train, xs_test, ys_train, ys_test = train_test_split(x, y, test_size=0.20, random_state=42)

In [16]:
# defining the classifiers
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

svm_clf = SVC(C=0.1)
nb_clf = MultinomialNB()
dt_clf = DecisionTreeClassifier()
mlp_clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

In [18]:
#defining the voting mechanism
from sklearn.ensemble import VotingClassifier

voting_clf = VotingClassifier(
    estimators=[('svm', svm_clf), ('nb', nb_clf), ('df', dt_clf), ('mlp', mlp_clf)], voting='hard')

In [19]:
voting_clf.fit(xs_train, ys_train)

VotingClassifier(estimators=[('svm', SVC(C=0.1)), ('nb', MultinomialNB()),
                             ('df', DecisionTreeClassifier()),
                             ('mlp',
                              MLPClassifier(alpha=1e-05,
                                            hidden_layer_sizes=(5, 2),
                                            random_state=1, solver='lbfgs'))])

In [20]:
for clf in (svm_clf, nb_clf, dt_clf, mlp_clf):
    clf.fit(xs_train, ys_train)

In [21]:
# measure the train and test accuracy of each classifier, including the ensembled one
from sklearn.metrics import accuracy_score

for clf in (svm_clf, nb_clf, dt_clf, mlp_clf, voting_clf):
    print(clf.__class__.__name__)
    ys_pred = clf.predict(xs_train)
    print('\ttrain:', accuracy_score(ys_train, ys_pred))
    ys_pred = clf.predict(xs_test)
    print('\ttest:', accuracy_score(ys_test, ys_pred))

SVC
	train: 0.41124260355029585
	test: 0.34911242603550297
MultinomialNB
	train: 0.4437869822485207
	test: 0.41420118343195267
DecisionTreeClassifier
	train: 1.0
	test: 0.7337278106508875
MLPClassifier
	train: 0.2603550295857988
	test: 0.2485207100591716
VotingClassifier
	train: 0.6405325443786982
	test: 0.5621301775147929
